# Sequence Analysis with Python

Contact: Veli Mäkinen veli.makinen@helsinki.fi

The following assignments introduce applications of hashing with $\verb+dict()+$ primitive of Python. While doing so, a rudimentary introduction to biological sequences is given. 
This framework is then enhanced with probabilities, leading to routines to generate random sequences under some constraints, including a general concept of *Markov-chains*. All these components illustrate the usage of $\verb+dict()+$, but at the same time introduce some other computational routines to efficiently deal with probabilities.   
The function $\verb+collections.defaultdict+$ can be useful.

## DNA and RNA

A DNA molecule consist, in principle, of a chain of smaller molecules. These smaller molecules have some common basic components (bases) that repeat. For our purposes it is sufficient to know that these bases are nucleotides adenine, cytosine, guanine, and thymine with abbreviations $\texttt{A}$, $\texttt{C}$, $\texttt{G}$, and $\texttt{T}$. Given a *DNA sequence* e.g. $\texttt{ACGATGAGGCTCAT}$, one can reverse engineer (with negligible loss of information) the corresponding DNA molecule.

Parts of a DNA molecule can *transcribe* into an RNA molecule. In this process, thymine gets replaced by uracil ($\texttt{U}$). 


1. Write a function $\verb+dna_to_rna+$ to convert a DNA sequence into an RNA sequence. For the sake of exercise, use $\verb+dict()+$ to store the symbol to symbol encoding rules. Create a program to test your function.

In [1]:
from IPython.core import page
page.page=print # turn off the pager, so that %pycat prints inline

In [15]:
# exercise 1 DO NOT MODIFY THIS LINE
%pycat part07-e01_dna_to_rna/src/dna_to_rna.py

#!/usr/bin/env python3

DNA_TO_RNA = {
    "C": "C",
    "G": "G",
    "A": "A",
    "T": "U"
}

def dna_to_rna(s):
    as_rna = s
    for dna_base, rna_base in DNA_TO_RNA.items():
        as_rna = as_rna.replace(dna_base, rna_base)
    return as_rna



### Idea of solution
The iterates over the RNA to DNA mappings dict, replacing the DNA nucleotide with the RNA nucleotide one by one. 

### Discussion
The solution works as expected, essentially replacing each T with a U since this is only difference between DNA and RNA.

## Proteins

Like DNA and RNA, protein molecule can be interpreted as a chain of smaller molecules, where the bases are now amino acids. RNA molecule may *translate* into a protein molecule, but instead of base by base, three bases of RNA correspond to one base of protein. That is, RNA sequence is read triplet (called codon) at a time. 

2. Consider the codon to amino acid conversion table in http://www.kazusa.or.jp/codon/cgi-bin/showcodon.cgi?species=9606&aa=1&style=N. Write a function $\verb+get_dict+$ to read the table into a $\verb+dict()+$, such that for each RNA sequence of length 3, say $\texttt{AGU}$, the hash table stores the conversion rule to the corresponding amino acid. You may store the html page to your local src directory,
and parse that file.

In [17]:
# exercise 2 DO NOT MODIFY THIS LINE
%pycat part07-e02_rna_to_prot/src/rna_to_prot.py

#!/usr/bin/env python3
from bs4 import BeautifulSoup
import re

CODON_REGEX = r'([A-Z]{3})\s+'
AA_REGEX = r'\s+([A-Z*]{1})\s+'

def get_path(filename):
    import sys
    import os
    return os.path.join(os.path.dirname(sys.argv[0]), "..", "src", filename)

def get_dict():
    html = open(get_path("Codon_usage_table.html")).read()
    parsed = BeautifulSoup(html, 'html.parser').pre.text
    codons = re.findall(CODON_REGEX, parsed)
    aminoacids = re.findall(AA_REGEX, parsed)

    return dict(zip(codons, aminoacids))

if __name__ == '__main__':
    codon_to_aa = get_dict()
    print(codon_to_aa)



### Idea of solution

Note: I renamed the html file not contain whitespace as usually whitespace in file names used in code is usually a very bad idea. 

The solution uses BeautifulSoup parser library to parse the document and  and extract the contents of pre tag. The it uses regexes to extract the necessary data. In the case of the codon the regex expects 3 consecutive alphanumeric letters flanked by whitespace and in the case of aminoacid the requirement is the same expect only one letter is allowed. 

After that the two lists are zipped into a dict. 


### Discussion
The function works as expected returning a dict which has the length 64 where the codons are the keys and aminoacids are the values.  

The result:

{'UUU': 'F', 'UCU': 'S', 'UAU': 'Y', 'UGU': 'C', 'UUC': 'F', 'UCC': 'S', 'UAC': 'Y', 'UGC': 'C', 'UUA': 'L', 'UCA': 'S', 'UAA': '*', 'UGA': '*', 'UUG': 'L', 'UCG': 'S', 'UAG': '*', 'UGG': 'W', 'CUU': 'L', 'CCU': 'P', 'CAU': 'H', 'CGU': 'R', 'CUC': 'L', 'CCC': 'P', 'CAC': 'H', 'CGC': 'R', 'CUA': 'L', 'CCA': 'P', 'CAA': 'Q', 'CGA': 'R', 'CUG': 'L', 'CCG': 'P', 'CAG': 'Q', 'CGG': 'R', 'AUU': 'I', 'ACU': 'T', 'AAU': 'N', 'AGU': 'S', 'AUC': 'I', 'ACC': 'T', 'AAC': 'N', 'AGC': 'S', 'AUA': 'I', 'ACA': 'T', 'AAA': 'K', 'AGA': 'R', 'AUG': 'M', 'ACG': 'T', 'AAG': 'K', 'AGG': 'R', 'GUU': 'V', 'GCU': 'A', 'GAU': 'D', 'GGU': 'G', 'GUC': 'V', 'GCC': 'A', 'GAC': 'D', 'GGC': 'G', 'GUA': 'V', 'GCA': 'A', 'GAA': 'E', 'GGA': 'G', 'GUG': 'V', 'GCG': 'A', 'GAG': 'E', 'GGG': 'G'}

3. Use the same conversion table as above, but now write function $\verb+get_dict+$ to read the table into a $\verb+dict()+$, such that for each amino acid the hash table stores the list of codons encoding it.    

In [18]:
# exercise 3 DO NOT MODIFY THIS LINE

%pycat part07-e03_prot_to_rna/src/prot_to_rna.py


#!/usr/bin/env python3
import re
from collections import defaultdict
from bs4 import BeautifulSoup

CODON_REGEX = r'([A-Z]{3})\s+'
AA_REGEX = r'\s+([A-Z*]{1})\s+'

def get_path(filename):
    import sys
    import os
    return os.path.join(os.path.dirname(sys.argv[0]), "..", "src", filename)

def get_dict():
    html = open(get_path("Codon_usage_table.html")).read()
    parsed = BeautifulSoup(html, 'html.parser').pre.text
    codons = re.findall(CODON_REGEX, parsed)
    aminoacids = re.findall(AA_REGEX, parsed)

    aa_to_codon_dict = defaultdict(list)
    for index, aa in enumerate(aminoacids):
        aa_to_codon_dict[aa].append(codons[index])

    return dict(aa_to_codon_dict)

if __name__ == '__main__':
    aa_to_codons = get_dict()
    print(aa_to_codons)



### Idea of solution

The parsing part of the function works as described above. The returned dictionary is formed in a loop where the codon is appended to a list for the codon that encodes it. This loop code is made slightly more succinct by using Python defaultdict which means handles the case where the codon list is still empty and initializes it. 

### Discussion

The result:
{'F': ['UUU', 'UUC'], 'S': ['UCU', 'UCC', 'UCA', 'UCG', 'AGU', 'AGC'], 'Y': ['UAU', 'UAC'], 'C': ['UGU', 'UGC'], 'L': ['UUA', 'UUG', 'CUU', 'CUC', 'CUA', 'CUG'], '*': ['UAA', 'UGA', 'UAG'], 'W': ['UGG'], 'P': ['CCU', 'CCC', 'CCA', 'CCG'], 'H': ['CAU', 'CAC'], 'R': ['CGU', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'], 'Q': ['CAA', 'CAG'], 'I': ['AUU', 'AUC', 'AUA'], 'T': ['ACU', 'ACC', 'ACA', 'ACG'], 'N': ['AAU', 'AAC'], 'K': ['AAA', 'AAG'], 'M': ['AUG'], 'V': ['GUU', 'GUC', 'GUA', 'GUG'], 'A': ['GCU', 'GCC', 'GCA', 'GCG'], 'D': ['GAU', 'GAC'], 'G': ['GGU', 'GGC', 'GGA', 'GGG'], 'E': ['GAA', 'GAG']}

With the conversion tables at hand, the following should be trivial to solve.

4. Fill in function $\verb+dna_to_prot+$ in the stub solution to convert a DNA sequence into a protein sequence. 
You may copy-paste the result dictionaries from exercises 2 and 3.
Run your program with $\texttt{ATGATATCATCGACGATGTAG}$.   

In [19]:
# exercise 4 DO NOT MODIFY THIS LINE
%pycat part07-e04_dna_to_prot/src/dna_to_prot.py

#!/usr/bin/env python3
import re
RNA_TO_PROT = {'UUU': 'F', 'UCU': 'S', 'UAU': 'Y', 'UGU': 'C', 'UUC': 'F', 'UCC': 'S', 'UAC': 'Y', 'UGC': 'C', 'UUA': 'L', 'UCA': 'S', 'UAA': '*', 'UGA': '*', 'UUG': 'L', 'UCG': 'S', 'UAG': '*', 'UGG': 'W', 'CUU': 'L', 'CCU': 'P', 'CAU': 'H', 'CGU': 'R', 'CUC': 'L', 'CCC': 'P', 'CAC': 'H', 'CGC': 'R', 'CUA': 'L', 'CCA': 'P', 'CAA': 'Q', 'CGA': 'R', 'CUG': 'L', 'CCG': 'P', 'CAG': 'Q', 'CGG': 'R', 'AUU': 'I', 'ACU': 'T', 'AAU': 'N', 'AGU': 'S', 'AUC': 'I', 'ACC': 'T', 'AAC': 'N', 'AGC': 'S', 'AUA': 'I', 'ACA': 'T', 'AAA': 'K', 'AGA': 'R', 'AUG': 'M', 'ACG': 'T', 'AAG': 'K', 'AGG': 'R', 'GUU': 'V', 'GCU': 'A', 'GAU': 'D', 'GGU': 'G', 'GUC': 'V', 'GCC': 'A', 'GAC': 'D', 'GGC': 'G', 'GUA': 'V', 'GCA': 'A', 'GAA': 'E', 'GGA': 'G', 'GUG': 'V', 'GCG': 'A', 'GAG': 'E', 'GGG': 'G'}

DNA_TO_RNA = {
    "C": "C",
    "G": "G",
    "A": "A",
    "T": "U"
}

def dna_to_rna(s):
    as_rna = s
    for dna_base, rna_base in DNA_TO_RNA.items():
        as_rna = as_rna.re

### Idea of solution

The solution uses the dna_to_rna function which was implemented in exercise 1. Once RNA is produced, the rna_to_prot first splits the DNA string to codons by using a regex which are then mapped to amino acids by using the conversion dict produced in the previous exercise. If the input string length is not divisible by three, the excess letters in the end are ignored. 

### Discussion
The result for dna_to_prot("ATGATATCATCGACGATGTAG") is MISSTM*.

You may notice that there are $4^3=64$ different codons, but only 20 amino acids. That is, some triplets encode the same amino acid.  

## Reverse translation

It has been observed that among the codons coding the same amino acid, some are more frequent than others. These frequencies can be converted to probabilities. E.g. consider codons $\texttt{AUU}$, $\texttt{AUC}$, and $\texttt{AUA}$ that code for amino acid isoleucine.
If they are observed, say, 36, 47, 17 times, respectively, to code isoleucine in a dataset, the probability that a random such event is $\texttt{AUU}$ $\to$ isoleucine is 36/100.

This phenomenon is called *codon adaptation*, and for our purposes it works as a good introduction to generation of random sequences under constraints.   

5. Consider the codon adaptation frequencies in http://www.kazusa.or.jp/codon/cgi-bin/showcodon.cgi?species=9606&aa=1&style=N and read them into a $\verb+dict()+$, such that for each RNA sequence of length 3, say $\texttt{AGU}$, the hash table stores the probability of that codon among codons encoding the same amino acid.
Put your solution in the $\verb+get_dict+$ function.

In [14]:
# exercise 5 DO NOT MODIFY THIS LINE
%pycat part07-e05_codon_to_prob/src/codon_to_prob.py

#!/usr/bin/env python3
from bs4 import BeautifulSoup
import re
from collections import defaultdict
import pandas as pd
CODON_REGEX = r'([A-Z]{3})\s+'
AA_REGEX = r'\s+([A-Z*]{1})\s+'
FREQ_REGEX = r'(\d+.\d)\s{1}\('

def get_path(filename):
    import sys
    import os
    return os.path.join(os.path.dirname(sys.argv[0]), "..", "src", filename)

def get_synonomous_freqs_sum(aa, df):
    synonymous_codons = df[df.aa == aa]
    return synonymous_codons.freqs_per_1000.sum()

def get_dict():
    html = open(get_path("Codon_usage_table.html")).read()
    codon_html = BeautifulSoup(html, 'html.parser').pre.text

    df = pd.DataFrame()
    df['codon'] = re.findall(CODON_REGEX,  codon_html)
    df['freqs_per_1000'] = list(map(float, re.findall(FREQ_REGEX, codon_html)))
    df['aa'] = re.findall(AA_REGEX, codon_html)
    df['syn_sum'] = df.aa.map(lambda aa: get_synonomous_freqs_sum(aa, df))
    df['freq_frac'] = round(df.freqs_per_1000 / df.syn_sum, 2)
    return dict(zip(df.codon, df.freq_frac)

### Idea of solution

fill in

### Discussion
fill in

Now you should have everything in place to easily solve the following.


6. Write a class $\verb+ProteinToMaxRNA+$ with a $\verb+convert+$ method which converts a protein sequence into the most likely RNA sequence to be the source of this protein. Run your program with $\texttt{LTPIQNRA}$.

In [24]:
# exercise 6 DO NOT MODIFY THIS LINE
%pycat part07-e06_protein_to_max_rna/src/protein_to_max_rna.py

#!/usr/bin/env python3

CODON_PROBS = {'UUU': 0.46, 'UCU': 0.19, 'UAU': 0.44, 'UGU': 0.46, 'UUC': 0.54, 'UCC': 0.22, 'UAC': 0.56, 'UGC': 0.54, 'UUA': 0.08, 'UCA': 0.15, 'UAA': 0.29, 'UGA': 0.47, 'UUG': 0.13, 'UCG': 0.05, 'UAG': 0.24, 'UGG': 1.0, 'CUU': 0.13, 'CCU': 0.29, 'CAU': 0.42, 'CGU': 0.08, 'CUC': 0.2, 'CCC': 0.32, 'CAC': 0.58, 'CGC': 0.18, 'CUA': 0.07, 'CCA': 0.28, 'CAA': 0.26, 'CGA': 0.11, 'CUG': 0.4, 'CCG': 0.11, 'CAG': 0.74, 'CGG': 0.2, 'AUU': 0.36, 'ACU': 0.25, 'AAU': 0.47, 'AGU': 0.15, 'AUC': 0.47, 'ACC': 0.36, 'AAC': 0.53, 'AGC': 0.24, 'AUA': 0.17, 'ACA': 0.28, 'AAA': 0.43, 'AGA': 0.22, 'AUG': 1.0, 'ACG': 0.11, 'AAG': 0.57, 'AGG': 0.21, 'GUU': 0.18, 'GCU': 0.27, 'GAU': 0.46, 'GGU': 0.16, 'GUC': 0.24, 'GCC': 0.4, 'GAC': 0.54, 'GGC': 0.34, 'GUA': 0.12, 'GCA': 0.23, 'GAA': 0.42, 'GGA': 0.25, 'GUG': 0.46, 'GCG': 0.11, 'GAG': 0.58, 'GGG': 0.25}
AA_TO_RNA = {'F': ['UUU', 'UUC'], 'S': ['UCU', 'UCC', 'UCA', 'UCG', 'AGU', 'AGC'], 'Y': ['UAU', 'UAC'], 'C': ['UGU', 'UGC'], 'L': ['UUA

### Idea of solution

The string is coverted to a list which is the mapped to the codon with the highest probability element-wise. The "private" __get_codon_with_max_prob__ helper method gets a list of codons, and subsets the the codon probabilities to contain only the ones in the parameter list. Then key with the max value is then returned.

### Discussion
The output produced for 'LTPIQNRA' is 'CUGACCCCCAUCCAGAACAGAGCC'.

Now we are almost ready to produce random RNA sequences that code a given protein sequence. For this, we need a subroutine to *sample from a probability distribution*. Consider our earlier example of probabilities 36/100, 47/100, and 17/100 for $\texttt{AUU}$, $\texttt{AUC}$, and $\texttt{AUA}$, respectively. 
Let us assume we have a random number generator $\verb+random()+$ that returns a random number from interval $[0,1)$. We may then partition the unit interval according to cumulative probabilities to [0,36/100), [36/100,83/100), [83/100,1), respectively. Depending which interval the number $\verb+random()+$ hits, we select the codon accordingly.

7. Write a function $\verb+random_event+$ that chooses a random event, given a probability distribution (set of events whose probabilities sum to 1).
You can use function $\verb+random.uniform+$ to produce values uniformly at random from the range $[0,1)$. The distribution should be given to your function as a dictionary from events to their probabilities.

In [20]:
# exercise 7 DO NOT MODIFY THIS LINE
%pycat part07-e07_random_event/src/random_event.py

#!/usr/bin/env python3

import numpy as np


def random_event(dist):
    """Takes as input a dictionary from events to their probabilities.
Return a random event sampled according to the given distribution.
The probabilities must sum to 1.0"""
    # Write your solution here
    return np.random.choice(list(dist.keys()), 1, p=list(dist.values()))[0]

if __name__ == '__main__':
    distribution=dict(zip("ACGT", [0.10, 0.35, 0.15, 0.40]))
    print(random_event(distribution))



### Idea of solution

Solution uses the ready-made numpy __choice__ method.

### Discussion
fill in

With this general routine, the following should be easy to solve.
 
8. Write a class $\verb+ProteinToRandomRNA+$ to produce a random RNA sequence encoding the input protein sequence according to the input codon adaptation probabilities. The actual conversion is done through the $\verb+convert+$ method. Run your program with $\texttt{LTPIQNRA}$.

In [12]:
# exercise 8 DO NOT MODIFY THIS LINE
%pycat part07-e08_protein_to_random_rna/src/protein_to_random_rna.py

#!/usr/bin/env python3

import random


def random_event(dist):
    """Takes as input a dictionary from events to their probabilities.
Return a random event sampled according to the given distribution.
The probabilities must sum to 1.0"""
    Copy the solution from the previous exercise here
    return event

class ProteinToRandomRNA(object):
    def __init__(self):

    def convert(self, s):
        return ""
        
if __name__ == '__main__':
    import sys
    protein_to_random_codons = ProteinToRandomRNA()
    if len(sys.argv) == 1:
        print(protein_to_random_codons.convert("LTPIQNRA"))
    for s in sys.argv[1:]:
        print(protein_to_random_codons.convert(s))



### Idea of solution

fill in

### Discussion
fill in

## Generating DNA sequences with higher-order Markov chains

We will now reuse the machinery derived above in a related context. We go back to DNA sequences, and consider some easy statistics that can be used to characterize the sequences. 
First, just the frequencies of bases $\texttt{A}$, $\texttt{C}$, $\texttt{G}$, $\texttt{T}$ may reveal the species from which the input DNA originates; each species has a different base composition that has been formed during evolution. 
More interestingly, the areas where DNA to RNA transcription takes place (coding region) have an excess of $\texttt{C}$ and $\texttt{G}$ over $\texttt{A}$ and $\texttt{T}$. To detect such areas a common routine is to just use a *sliding window* of fixed size, say $k$, and compute for each window position 
$T[i..i+k-1]$ the base frequencies, where $T[1..n]$ is the input DNA sequence. When sliding the window from  $T[i..i+k-1]$ to $T[i+1..i+k]$ frequency $f(T[i])$ gets decreases by one and $f(T[i+k])$ gets increased by one. 

9. Write a *generator* $\verb+sliding_window+$ to compute sliding window base frequencies so that each moving of the window takes constant time. We saw in the beginning of the course one way how to create generators using
  generator expression. Here we use a different way. For the function $\verb+sliding_window+$ to be a generator, it must have at least
  one $\verb+yield+$ expression, see https://docs.python.org/3/reference/expressions.html#yieldexpr.
  See also exercise 13 for an example of a generator $\verb+all_kmers+$ made with yield expressions.
  A yield expression can be used to return a value and *temporarily* return from the function.

In [ ]:
# exercise 9 DO NOT MODIFY THIS LINE
%pycat part07-e09_sliding_window/src/sliding_window.py

### Idea of solution

fill in

### Discussion
fill in

 
Our models so far have been so-called *zero-order* models, as each event has been independent of other events. With sequences, the dependencies of events are naturally encoded by their *contexts*. Considering that a sequence is produced from left-to-right, a *first-order* context for $T[i]$ is $T[i-1]$, that is, the immediately preceding symbol. *First-order Markov chain* is a sequence produced by generating $c=T[i]$ with the probability of event of seeing symbol $c$ after previously generated symbol $a=T[i-1]$. The first symbol of the chain is sampled according to the zero-order model.  
The first-order model can naturally be extended to contexts of length $k$, with $T[i]$ depending on $T[i-k..i-1]$. Then the first $k$ symbols of the chain are sampled according to the zero-order model.  The following assignments develop the routines to work with the *higher-order Markov chains*. 
In what follows, a $k$-mer is a substring $T[i..i+k-1]$ of the sequence at an arbitrary position. 

10. Write function $\verb+context_list+$ that given an input DNA sequence $T$ associates to each $k$-mer $W$ the concatenation of all symbols $c$ that appear after context $W$ in $T$, that is, $T[i..i+k]=Wc$. For example, <span style="color:red; font:courier;">GA</span> is associated to <span style="color:blue; font: courier;">TCT</span> in $T$=<span style="font: courier;">AT<span style="color:red;">GA</span><span style="color:blue;">T</span>ATCATC<span style="color:red;">GA</span><span style="color:blue;">C</span><span style="color:red;">GA</span><span style="color:blue;">T</span>GTAG</span>, when $k=2$.

In [23]:
# exercise 10 DO NOT MODIFY THIS LINE
%pycat part07-e10_context_list/src/context_list.py

#!/usr/bin/env python3

import sys
from collections import defaultdict

def kmer_gen(string, k):
    idx = 0
    while idx < len(string) - k:
        kmer = string[idx:idx+k]
        next_n = string[idx+k]
        idx += 1
        yield (kmer, next_n)

def context_list(s, k):
    kmers = defaultdict(str)
    for kmer, next_n in kmer_gen(s, k):
        kmers[kmer] = kmers[kmer] + next_n
    return kmers

if __name__ == '__main__':
    k=2
    if len(sys.argv) == 1:
        d=context_list("ATGATATCATCGACGATCTAG", k)
        print(d)
    else:
        for s in sys.argv[1:]:
            d=context_list(s, k)
            print(d)



### Idea of solution

fill in

### Discussion
The output for context_list("ATGATATCATCGACGATCTAG", 2) is:

{'AT': 'GACCC', 'TG': 'A', 'GA': 'TCT', 'TA': 'TG', 'TC': 'AGT', 'CA': 'T', 'CG': 'AA', 'AC': 'G', 'CT': 'A'}

11. With the above solution, write function $\verb+context_probabilities+$ to count the frequencies of symbols in each context and convert these frequencies into probabilities. Run your program with $T=\texttt{ATGATATCATCGACGATGTAG}$ and $k$ values 0 and 2.

In [ ]:
# exercise 11 DO NOT MODIFY THIS LINE
%pycat part07-e11_context_probabilities/src/context_probabilities.py

### Idea of solution

fill in

### Discussion
The output for context_probabilities("ATGATATCATCGACGATGTAG", 2) is: 

{'AT': {'A': 0.2, 'T': 0.0, 'G': 0.4, 'C': 0.4}, 'TG': {'A': 0.5, 'T': 0.5, 'G': 0.0, 'C': 0.0}, 'GA': {'A': 0.0, 'T': 0.6666666666666666, 'G': 0.0, 'C': 0.3333333333333333}, 'TA': {'A': 0.0, 'T': 0.5, 'G': 0.5, 'C': 0.0}, 'TC': {'A': 0.5, 'T': 0.0, 'G': 0.5, 'C': 0.0}, 'CA': {'A': 0.0, 'T': 1.0, 'G': 0.0, 'C': 0.0}, 'CG': {'A': 1.0, 'T': 0.0, 'G': 0.0, 'C': 0.0}, 'AC': {'A': 0.0, 'T': 0.0, 'G': 1.0, 'C': 0.0}, 'GT': {'A': 1.0, 'T': 0.0, 'G': 0.0, 'C': 0.0}}

12. With the above solution and the function $\verb+random_event+$ from the earlier exercise, write class $\verb+MarkovChain+$. Its $\verb+generate+$ method should generate a random DNA sequence following the original $k$-th order Markov chain probabilities. 

In [ ]:
# exercise 12 DO NOT MODIFY THIS LINE
%pycat part07-e12_generate_markov/src/generate_markov.py

### Idea of solution

fill in

### Discussion
fill in

If you have survived so far without problems, please run your program a few more times with different inputs. At some point you should get a lookup error in your hash-table! The reason for this is not your code, but the way we defined the model: Some $k$-mers may not be among the training data (input sequence $T$), but such can be generated as the first $k$-mer that is generated using the zero-order model.  

A general approach to fixing such issues with incomplete training data is to use *pseudo counts*. That is, all imaginable events are initialized to frequency count 1.   

13. Modify the previous solution 11 to use pseudo counts in order to obtain a $k$-th order Markov chain that can assign a probability for any DNA sequence. 
You may use the provided generator $\verb+all_kmers+$ to iterate over all $k$-mer of given length.

In [ ]:
# exercise 13 DO NOT MODIFY THIS LINE
%pycat part07-e13_pseudocounts/src/pseudocounts.py

### Idea of solution

fill in

### Discussion

Output for context_probabilities("ATGATATCATCGACGATGTAG", 2) is:

{'AA': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
 'AC': {'A': 0.2, 'C': 0.2, 'G': 0.4, 'T': 0.2},
 'AG': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
 'AT': {'A': 0.222, 'C': 0.333, 'G': 0.333, 'T': 0.111},
 'CA': {'A': 0.2, 'C': 0.2, 'G': 0.2, 'T': 0.4},
 'CC': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
 'CG': {'A': 0.5, 'C': 0.167, 'G': 0.167, 'T': 0.167},
 'CT': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
 'GA': {'A': 0.143, 'C': 0.286, 'G': 0.143, 'T': 0.429},
 'GC': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
 'GG': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25},
 'GT': {'A': 0.4, 'C': 0.2, 'G': 0.2, 'T': 0.2},
 'TA': {'A': 0.167, 'C': 0.167, 'G': 0.333, 'T': 0.333},
 'TC': {'A': 0.333, 'C': 0.167, 'G': 0.333, 'T': 0.167},
 'TG': {'A': 0.333, 'C': 0.167, 'G': 0.167, 'T': 0.333},
 'TT': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}}

14. Write class $\verb+MarkovChain+$ that given the $k$-th order Markov chain developed above to the constructor, its method $\verb+probability+$ computes the probability of a given input DNA sequence.

In [ ]:
# exercise 14 DO NOT MODIFY THIS LINE
%pycat part07-e14_markov_chain_probability/src/markov_chain_probability.py

### Idea of solution

fill in

### Discussion
fill in

With the last assignment you might end up in trouble with precision, as multiplying many small probabilities gives a really small number in the end. There is an easy fix by using so-called log-transform. 
Consider computation of $P=s_1 s_2 \cdots s_n$, where $0\leq s_i\leq 1$ for each $i$. Taking logarithm in base 2 from both sides gives $\log _2 P= \log _2 (s_1 s_2 \cdots s_n)=\log_2 s_1 + \log_2 s_2 + \cdots \log s_n= \sum_{i=1}^n \log s_i$, with repeated application of the property that the logarithm of a multiplication of two numbers is the sum of logarithms of the two numbers taken separately. The results is abbreviated as log-probability.

15. Write class $\verb+MarkovChain+$ that given the $k$-th order Markov chain developed above to the constructor, its method $\verb+log_probability+$ computes the log-probability of a given input DNA sequence. Run your program with $T=\texttt{ATGATATCATCGACGATGTAG}$ and $k=2$.

In [ ]:
# exercise 15 DO NOT MODIFY THIS LINE
%pycat part07-e15_markov_chain_log_probability/src/markov_chain_log_probability.py

### Idea of solution

fill in

### Discussion
fill in

Finally, if you try to use the code so far for very large inputs, you might observe that the concatenation of symbols following a context occupy considerable amount of space. This is unnecessary, as we only need the frequencies. 

16. Optimize the space requirement of your code from exercise 13 for the $k$-th order Markov chain by replacing the concatenations by direct computations of the frequencies. Implement this as the
  $\verb+context_probabilities+$ function.

In [ ]:
# exercise 16 DO NOT MODIFY THIS LINE
%pycat part07-e16_low_space_requirement/src/low_space_requirement.py

### Idea of solution

fill in

### Discussion
fill in

While the earlier approach of explicit concatenation of symbols following a context suffered from inefficient use of space, it does have a benefit of giving another much simpler strategy to sample from the distribution: 
observe that an element of the concatenation taken uniformly randomly is sampled exactly with the correct probability. 

17. Revisit the solution 12 and modify it to directly sample from the concatenation of symbols following a context. You may use the function $\verb+random.choice+$.

In [ ]:
# exercise 17 DO NOT MODIFY THIS LINE
%pycat part07-e17_sample_from_concatenation/src/sample_from_concatenation.py

### Idea of solution

fill in

### Discussion
fill in

## $k$-mer index

Our $k$-th order Markov chain can now be modified to a handy index structure called $k$-mer index. This index structure associates to each $k$-mer its list of occurrence positions in DNA sequence $T$.  Given a query $k$-mer $W$, one can thus easily list all positions $i$ with  $T[i..k-1]=W$.

18. Implement function $\verb+kmer_index+$ by modifying your earlier code for the $k$-th order Markov chain. Test your program with $T=\texttt{ATGATATCATCGACGATGTAG}$ and $k=2$.

In [ ]:
# exercise 18 DO NOT MODIFY THIS LINE
%pycat part07-e18_kmer_index/src/kmer_index.py

### Idea of solution

fill in

### Discussion
fill in

## Comparison of probability distributions

Now that we know how to learn probability distributions from data, we might want to compare two such distributions, for example, to test if our programs work as intended. 

Let $P=\{p_1,p_2,\ldots, p_n\}$ and $Q=\{q_1,q_2,\ldots, q_n\}$ be two probability distributions for the same set of $n$ events. This means $\sum_{i=1}^n p_i=\sum_{i=1}^n q_i=1$, $0\leq p_j \leq 1$, and $0\leq q_j \leq 1$ for each event $j$. 

*Kullback-Leibler divergence* is a measure $d()$ for the *relative entropy* of $P$ with respect to $Q$ defined as 
$d(P|Q)=\sum_{i=1}^n p_i \log\frac{p_i}{q_i}$.


This measure is always non-negative, and 0 only when $P=Q$. It can be interpreted as the gain of knowing $Q$ to encode $P$. Note that this measure is not symmetric.

19. Write function $\verb+kullback_leibler+$ to compute $d(P||Q)$. Test your solution by generating a random RNA sequence
  encoding the input protein sequence according to the input codon adaptation probabilities.
  Then you should learn the codon adaptation probabilities from the RNA sequence you generated.
  Then try the same with uniformly random RNA sequences (which don't have to encode any
  specific protein sequence). Compute the relative entropies between the
  three distribution (original, predicted, uniform) and you should observe a clear difference.
  Because $d(P||Q)$ is not symmetric, you can either print both $d(P||Q)$ and $d(Q||P)$,
  or their average.

In [ ]:
# exercise 19 DO NOT MODIFY THIS LINE
%pycat part07-e19_kullback_leibler/src/kullback_leibler.py

### Idea of solution

fill in

### Discussion
fill in

## Stationary and equilibrium distributions (extra)

Let us consider a Markov chain of order one on the set of nucleotides.
Its transition probabilities can be expressed as a $4 \times 4$ matrix
$P=(p_{ij})$, where the element $p_{ij}$ gives the probability of the $j$th nucleotide
on the condition the previous nucleotide was the $i$th. An example of a transition matrix
is
$$\begin{tabular}{l|rrrr}
{} &     A &    C &     G &    T \\
\hline
A &  0.30 &  0.0 &  0.70 &  0.0 \\
C &  0.00 &  0.4 &  0.00 &  0.6 \\
G &  0.35 &  0.0 &  0.65 &  0.0 \\
T &  0.00 &  0.2 &  0.00 &  0.8 \\
\end{tabular}.$$

A distribution $\pi=(\pi_1,\pi_2,\pi_3,\pi_4)$ is called *stationary*, if
$\pi = \pi P$ (the product here is matrix product).

20. Write function $\verb+get_stationary_distributions+$ that gets a transition matrix as parameter,
  and returns the list of stationary distributions. You can do this with NumPy by
  first taking transposition of both sides of the above equation to get equation
  $\pi^T = P^T \pi^T$. Using numpy.linalg.eig take all eigenvectors related to
  eigenvalue 1.0. By normalizing these vectors to sum up to one get the stationary distributions
  of the original transition matrix. In the $\verb+main+$ function print the stationary distributions
  of the above transition matrix.

In [ ]:
# exercise 20 DO NOT MODIFY THIS LINE
%pycat part07-e20_stationary_distribution/src/stationary_distribution.py

### Idea of solution

fill in

### Discussion
fill in

21. Implement the following in the $\verb+main+$ function.
  Use again the above transition matrix.
  Choose one stationary distribution as the initial distribution of the Markov chain.
  Using your modified Markov chain generator generate a nucleotide sequence $s$ of length 10~000.
  Choose prefixes of $s$ of lengths 1, 10, 100, 1000, and 10 000. For each of these
  prefixes find out their nucleotide distribution (of order 0) using your earlier tool.
  Use 1 as the pseudo count.
  Then, for each prefix, compute the KL divergence between the initial distribution and the normalized
  nucleotide distribution.

In [ ]:
# exercise 21 DO NOT MODIFY THIS LINE
%pycat part07-e21_stationary_initial_distribution/src/stationary_initial_distribution.py

### Idea of solution

fill in

### Discussion
fill in

22. Implement the following in the $\verb+main+$ function.
Find the stationary distribution for the following transition matrix:  
$$\begin{tabular}{l|rrrr}
{} &     A &     C &     G &     T \\
\hline
A &  0.30 &  0.10 &  0.50 &  0.10 \\
C &  0.20 &  0.30 &  0.15 &  0.35 \\
G &  0.25 &  0.15 &  0.20 &  0.40 \\
T &  0.35 &  0.20 &  0.40 &  0.05 \\
\end{tabular}$$

Since there is only one stationary distribution, it is called the *equilibrium distribution*.
Choose randomly two nucleotide distributions. You can take these from your sleeve or
sample them from the Dirichlet distribution. Then for each of these distributions
as the initial distribution of the Markov chain, repeat the above experiment.
The state distribution should converge to the equilibrium distribution no matter how we
start the Markov chain! 

In [ ]:
# exercise 22 DO NOT MODIFY THIS LINE
%pycat part07-e22_equilibrium_distribution/src/equilibrium_distribution.py

### Idea of solution

fill in

### Discussion
fill in